In [ ]:
#Please install the package below for tweet-processing.
#I removed all the retweet, URL, @username and hashtag from the tweets 

#pip install tweet-preprocessor

In [53]:
import json
import preprocessor as p
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

In [29]:
tweets = []
for line in open('/Users/chidera/Desktop/block_6/COLX_585/US_data_about_covid.json.txt', 'r'):
    tweets.append(json.loads(line))

In [48]:
COVID_19_preprocessed = []
for tweet in tweets:
    COVID_19_preprocessed.append(p.clean(tweet['tweet_text']))
    
COVID_19_preprocessed_df = pd.DataFrame(COVID_19_preprocessed, columns=['preprocessed_tweets'])

In [31]:
# The non COVID_19 dataset
tweet_hate_df = pd.read_csv('/Users/chidera/Desktop/block_6/COLX_585/train_tweet_hate.csv')

#preprocessing the tweets 
tweet_hate_df['preprocessed'] = tweet_hate_df['tweet'].apply(lambda x: p.clean(x))

In [32]:
tweet_hate_df.head()

id  label                                              tweet  \
0   1      0   @user when a father is dysfunctional and is s...   
1   2      0  @user @user thanks for #lyft credit i can't us...   
2   3      0                                bihday your majesty   
3   4      0  #model   i love u take with u all the time in ...   
4   5      0             factsguide: society now    #motivation   

                                        preprocessed  
0  when a father is dysfunctional and is so selfi...  
1  thanks for credit i can't use cause they don't...  
2                                bihday your majesty  
3  i love u take with u all the time in urð±!!!...  
4                            factsguide: society now

In [49]:
# Before splitting, Write to a csv file 
tweet_hate_df.to_csv('tweet_hate_preprocessed.csv')
# keep only words
tweet_hate_df['preprocessed'] = tweet_hate_df['preprocessed'].str.replace("[^a-zA-Z\s]", "")
COVID_19_preprocessed_df.str.replace("[^a-zA-Z\s]", "").to_csv('COVID_19_preprocessed.csv')

In [ ]:
# drop unrelated columns, we only need 'preprocessed' and 'label'
tweet_hate_df = tweet_hate_df.drop(['Unnamed: 0', 'id','tweet'],1)
tweet_hate_df = tweet_hate_df[['preprocessed','label']]

# keep only words
tweet_hate_df['preprocessed'] = tweet_hate_df['preprocessed'].str.replace("[^a-zA-Z\s]", "")

# split data for training and testing
tweet_hate_df_dev = tweet_hate_df.iloc[31000:,:].reset_index(drop = True)
tweet_hate_df_train = tweet_hate_df.iloc[:31000,:].reset_index(drop = True)

# After split, write to a csv file, training and dev data 
tweet_hate_df_train.to_csv('tweet_hate_train.csv', index = False)
tweet_hate_df_dev.to_csv('tweet_hate_dev.csv', index = False)

In [ ]:
# test data
# combined the 1000 manully annotated label to the preprocessed tweets.
annotations = pd.read_excel('../data/annotated_covid.xlsx', header = None)
preprocessed = pd.read_csv('../data/COVID_19_preprocessed.csv')

# keep only tweet and annotation columns
tweet_hate_test = preprocessed.drop('Unnamed: 0',1)
tweet_hate_test = tweet_hate_test.iloc[:1000,:]

# keep only words
tweet_hate_test['preprocessed_tweets'] = tweet_hate_test['preprocessed_tweets'].str.replace("[^a-zA-Z\s]", "")
tweet_hate_test["label"] = annotations.iloc[:1000,1].astype(int)

# write to csv file
tweet_hate_test.to_csv('../data/tweet_hate_test.csv', index = False)

In [23]:
# Check the class balance
# using majority classification to calculate the baseline fscore
df['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [46]:
correct = 0
for i in range(len(gold_label)):
    if majority_class_label[i] == gold_label[i]:
        correct += 1
print('Accuracy: ', correct/len(gold_label))

Accuracy:  0.9298542018647143


In [57]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(gold_label, majority_class_label)
print('Average_precision: ', average_precision)

Average_precision:  0.07014579813528565


In [58]:
len(tweet_hate_df['label'])

majority_class_label = np.array([0]*31962)
gold_label = np.array(tweet_hate_df['label'])

print('Fscore: ', f1_score(gold_label, majority_class_label))

Fscore:  0.0
